In [1]:
from google.colab import drive
drive.mount('/content/gdrive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
cd /content/gdrive/My\ Drive/Project

/content/gdrive/My Drive/Project


In [32]:
import pandas as pd
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)




Using TensorFlow backend.


In [0]:
import csv
txt_file = r"all_analysis_data.txt"
csv_file = r"mycsv1.csv"

with open(txt_file) as fin, open(csv_file, 'w') as fout:
    o=csv.writer(fout,delimiter =",",)
    for line in fin:
        o.writerow(line.split())

In [0]:
data = pd.read_csv(csv_file,sep='\s+',names=["SYS_API_SEQ"])
data = data.reset_index()

In [58]:
data.head

<bound method NDFrame.head of       index                                        SYS_API_SEQ
0         0  ldrloaddll,ldrgetprocedureaddress,ldrloaddll,l...
1         1  getsystemtimeasfiletime,ntallocatevirtualmemor...
2         2  ldrgetdllhandle,ldrgetprocedureaddress,getsyst...
3         3  ldrloaddll,ldrgetprocedureaddress,ldrloaddll,l...
4         4  ldrloaddll,ldrgetprocedureaddress,ldrgetproced...
...     ...                                                ...
7102   7102  ldrloaddll,ldrgetprocedureaddress,ldrgetproced...
7103   7103  ldrloaddll,ldrgetprocedureaddress,ldrgetproced...
7104   7104  ldrloaddll,ldrgetprocedureaddress,ldrgetproced...
7105   7105  createthread,ntallocatevirtualmemory,ntfreevir...
7106   7106  ldrloaddll,ldrgetprocedureaddress,ldrgetproced...

[7107 rows x 2 columns]>

In [0]:
labels_data = pd.read_csv('labels.csv', delimiter=',',names=["Label"])
labels_data = labels_data.reset_index()


In [23]:
labels_data.head

<bound method NDFrame.head of       index     Label
0         0    Trojan
1         1    Trojan
2         2  Backdoor
3         3  Backdoor
4         4    Trojan
...     ...       ...
7102   7102     Virus
7103   7103     Virus
7104   7104     Virus
7105   7105     Virus
7106   7106     Virus

[7107 rows x 2 columns]>

In [0]:
train_data = pd.merge(data, labels_data, how='inner', on='index')

In [41]:
train_data.head

<bound method NDFrame.head of       index                                        SYS_API_SEQ     Label
0         0  ldrloaddll,ldrgetprocedureaddress,ldrloaddll,l...    Trojan
1         1  getsystemtimeasfiletime,ntallocatevirtualmemor...    Trojan
2         2  ldrgetdllhandle,ldrgetprocedureaddress,getsyst...  Backdoor
3         3  ldrloaddll,ldrgetprocedureaddress,ldrloaddll,l...  Backdoor
4         4  ldrloaddll,ldrgetprocedureaddress,ldrgetproced...    Trojan
...     ...                                                ...       ...
7102   7102  ldrloaddll,ldrgetprocedureaddress,ldrgetproced...     Virus
7103   7103  ldrloaddll,ldrgetprocedureaddress,ldrgetproced...     Virus
7104   7104  ldrloaddll,ldrgetprocedureaddress,ldrgetproced...     Virus
7105   7105  createthread,ntallocatevirtualmemory,ntfreevir...     Virus
7106   7106  ldrloaddll,ldrgetprocedureaddress,ldrgetproced...     Virus

[7107 rows x 3 columns]>

In [50]:
labels_data['Label'].nunique()

8

In [0]:
from sklearn.model_selection import train_test_split
y = train_data['Label']
train_data.drop('index', axis = 1, inplace = True)
x = train_data.drop('Label',axis=1)
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33, random_state=42)


In [59]:
print(X_train.shape)


(4761, 1)


In [0]:
model = Sequential()

model.add(LSTM(128, input_shape = X_train.shape, activation='relu', return_sequences=True))
model.add(Dense(32, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)
print(model.summary())

scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))